In [4]:
from knockouts import *

In [5]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# condition = "nonIBD"
condition = "IBD"
# condition = "UC"
# condition = "CD"
fluxdir = "./cases/fluxes/{}".format(condition)
overflowdir = "./cases/overflow/{}".format(condition)
if not os.path.isdir(fluxdir):
    os.makedirs(fluxdir)
if not os.path.isdir(overflowdir):
    os.makedirs(overflowdir)

In [ ]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/integration/solving_{}.log".format(condition), filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [ ]:
from overflow import get_overflow
def run(org):
    ListHandler.print_and_log("Loading {}".format(org))
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    index_dct = {r.id : model.reactions.index(r) for r in model.reactions}
    met_index_dct = {m.id : model.metabolites.index(m) for m in model.metabolites}
    me_nlp = get_nlp(model)
    max_mu = model.solution.objective_value*2
    
    del model
    # Run for all conditions
    killable = pandas.read_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None,index_col=0).index.to_list()
    for k in killable:
        knockout(index_dct,k,me_nlp,limit=lambda x:0)
    sol = optimize(index_dct,met_index_dct,me_nlp,max_mu=max_mu)
    # sol = model.solution.fluxes["biomass_dilution"] if hasattr(model,"solution") else "infeasible"
    ListHandler.print_and_log("Solution for {} : {}".format(org, sol))
    sol.to_frame().to_csv("cases/fluxes/{}/{}.csv".format(condition,org))
    # get_overflow(model).to_csv("./cases/overflow/{}/{}.csv".format(condition,org))

In [ ]:
survivors = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
processed = set(i.split("_killable")[0] for i in os.listdir("./ibdmdb/integration/{}".format(condition)) if "txt" in i)
done = set(i.split(".csv")[0] for i in os.listdir("./cases/fluxes/{}/".format(condition)) if "csv" in i)
run_for = processed - done
len(run_for)

In [ ]:
NP = min([10,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)
        
for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()